In [0]:
path_to_dataset = '/content/drive/My Drive/Machine Learning/dataset'
path_to_weights = '/content/drive/My Drive/Machine Learning'

In [0]:
import torchvision
import numpy as np

In [0]:
data = torchvision.datasets.MNIST(path_to_dataset,train=True,download=True)

In [0]:
from torch.utils.data import DataLoader,TensorDataset
import torch

In [0]:
x_train = np.array(list(map(lambda x : np.array(x[0]) , data)))
y_train = np.array(list(map(lambda x : x[1] , data)))


In [0]:
dataset = TensorDataset(torch.Tensor(x_train),torch.Tensor(y_train).long())
loader = DataLoader(dataset , 8)

## model

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [0]:
import torch.nn as nn

In [0]:
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        self.seq1 = nn.Sequential(
            nn.Conv2d(1,32,3),
            nn.ReLU(),
        )
        self.seq2 = nn.Sequential(
            nn.Linear(21632,256),
            nn.ReLU(),
            nn.Linear(256,64),
            nn.ReLU(),
            nn.Linear(64,10),
            nn.LogSoftmax(dim=1)
        )
        # self.c1 = nn.Conv2d(1,32,3)
        # self.L1 = nn.Linear(21632,256)
        # self.l2 = nn.Linear(256,64)
        # self.l3 = nn.Linear(64,10)
    def forward(self,x):
        # x = nn.functional.relu(self.c1(x))
        # x = nn.functional.relu(self.L1(x.view(-1,21632)))
        # x = nn.functional.relu(self.l2(x))
        # x = nn.functional.log_softmax(self.l3(x),dim=1)
        x = self.seq1(x)
        x = x.view(x.shape[0],-1)
        x = self.seq2(x)
        return x
    

In [0]:
sample_input = torch.Tensor(np.ndarray(28*28*2).reshape(2,1,28,28))

In [0]:
net = Net()
net = net.to(device)

In [0]:
# !pip install torchsummary

In [0]:
from torchsummary import summary

In [14]:
summary(net,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
              ReLU-2           [-1, 32, 26, 26]               0
            Linear-3                  [-1, 256]       5,538,048
              ReLU-4                  [-1, 256]               0
            Linear-5                   [-1, 64]          16,448
              ReLU-6                   [-1, 64]               0
            Linear-7                   [-1, 10]             650
        LogSoftmax-8                   [-1, 10]               0
Total params: 5,555,466
Trainable params: 5,555,466
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.34
Params size (MB): 21.19
Estimated Total Size (MB): 21.53
----------------------------------------------------------------


In [15]:
net

Net(
  (seq1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (seq2): Sequential(
    (0): Linear(in_features=21632, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): LogSoftmax()
  )
)

In [0]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adam(net.parameters())

In [17]:
Epochs = 3
total_batches = len(loader)
for epoc in range(Epochs):
    loss_aggr = 0
    print('\nStarting epoch {} of {}'.format(epoc,Epochs))
    for indx,(x,y) in enumerate(loader):
        if(indx%50==0):print('.',end='')
        optimizer.zero_grad()
        x = x.view(-1,1,28,28)
        x = x.to(device)
        y = y.to(device)
        out = net(x)
        loss = loss_fn(out,y)
        loss.backward()
        optimizer.step()
        loss+=loss.item()
    print('\nCompleted epoch {} with loss : {}'.format(epoc,loss))
    


Starting epoch 0 of 3
......................................................................................................................................................
Completed epoch 0 with loss : 0.0011754035949707031

Starting epoch 1 of 3
......................................................................................................................................................
Completed epoch 1 with loss : 0.00011134147644042969

Starting epoch 2 of 3
......................................................................................................................................................
Completed epoch 2 with loss : 0.0001647472381591797


In [19]:
#find accuracy
accuracy = 0
for inx,(x,y) in enumerate(loader):
    x = x.view(-1,1,28,28)
    x = x.to(device)
    y = y.to(device)
    out = net(x)
    # print(nn.functional.softmax(out,dim=1))
    ac = (out.argmax(dim=1)==y)
    accuracy += ac.sum().cpu().numpy()/(len(ac))
print(accuracy/len(loader))

0.987


In [0]:
import os
torch.save(net.state_dict(),os.path.join(path_to_weights,'mnist.pt'))